In [1]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Reshape, Flatten, Dropout, Activation
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from skimage.transform import resize
from imageio import imread

/home/aldopedraza/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/aldopedraza/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aldopedraza/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
path_data = '../all/'

In [3]:
def preprocess_input(np_img):
    np_img = np_img / 255
    return np_img

In [4]:
def generator(path_images, labels, batch_size, size_img = None):
    index_list = range(0, labels.shape[0])
    count = 0
    while True:
        batch_features = []
        batch_labels = []
        for i in range(batch_size):
            # Read image
            name_img = labels[count][0] + '.jpg'
            img = imread(path_images + '/' + name_img)
            
            # Resize the image
            if size_img != None:
                img = resize(img, (size_img, size_img))
            
            # Image preprocessing
            img = preprocess_input(img)

            label = labels[count][1:]

            batch_features.append(img)
            batch_labels.append(label)
            count += 1
            
            # Restart counter when it has reached the size 
            # of the data set
            if count == label.shape[0] - 1:
                count = 0
                break
            
        yield np.array(batch_features), np.array(batch_labels)
            

In [5]:
def galaxy_cnn(input_size, output_size):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_size))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(output_size, activation='sigmoid'))
    return model

In [6]:
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [7]:
labels_train = np.load(path_data + 'y_train.npy')
labels_test = np.load(path_data + 'y_test.npy')
labels_val = np.load(path_data + 'y_val.npy')

In [8]:
labels_val.shape

(9237, 38)

In [9]:
model = galaxy_cnn((424, 424, 3), 37)

In [10]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 422, 422, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 211, 211, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 209, 209, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 104, 104, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 104, 104, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 104, 104, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 102, 102, 128)     73856     
__________

In [11]:
model.compile(loss='mean_squared_error', optimizer='adagrad', metrics=[rmse])

In [12]:
batch_size = 8
steps = 43104 // batch_size 
steps_va = 9237 // batch_size
epochs = 10

In [13]:
model.fit_generator(generator(path_data + 'images_training_rev1', labels_train, batch_size=batch_size), 
                    steps_per_epoch=steps, 
                    epochs=epochs, 
                    validation_data= generator(path_data + 'images_training_rev1', labels_val, batch_size=batch_size),
                    validation_steps=steps_va)

Epoch 1/10
   3/5388 [..............................] - ETA: 32:55:17 - loss: 0.2015 - rmse: 0.4479

/home/aldopedraza/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.270563). Check your callbacks.
  % delta_t_median)


5388/5388 [==============================] - 901s 167ms/step - loss: 0.0594 - rmse: 0.2433 - val_loss: 0.0590 - val_rmse: 0.2427
Epoch 2/10
5388/5388 [==============================] - 878s 163ms/step - loss: 0.0582 - rmse: 0.2409 - val_loss: 0.0590 - val_rmse: 0.2427
Epoch 3/10
5388/5388 [==============================] - 863s 160ms/step - loss: 0.0574 - rmse: 0.2394 - val_loss: 0.0590 - val_rmse: 0.2427
Epoch 4/10
5388/5388 [==============================] - 863s 160ms/step - loss: 0.0536 - rmse: 0.2313 - val_loss: 0.0610 - val_rmse: 0.2465
Epoch 5/10
5388/5388 [==============================] - 862s 160ms/step - loss: 0.0491 - rmse: 0.2212 - val_loss: 0.0612 - val_rmse: 0.2470
Epoch 6/10
5388/5388 [==============================] - 863s 160ms/step - loss: 0.0453 - rmse: 0.2125 - val_loss: 0.0650 - val_rmse: 0.2545
Epoch 7/10
5388/5388 [==============================] - 863s 160ms/step - loss: 0.0448 - rmse: 0.2115 - val_loss: 0.0649 - val_rmse: 0.2544
Epoch 8/10
5388/5388 [=========

In [14]:
model.evaluate_generator(generator(path_data + 'images_training_rev1', labels_test, batch_size=batch_size), steps=steps)

[0.0531444140950089, 0.2299176670003274]